In [1]:
from urllib.request import urlopen

In [2]:
from bs4 import BeautifulSoup

In [3]:
import urllib.request, urllib.parse, urllib.error

### Monta downloads

In [4]:
emendas = urlopen("http://www.camara.gov.br/proposicoesWeb/prop_emendas?idProposicao=2122076&subst=0")

In [5]:
bsObje =  BeautifulSoup(emendas, "lxml")

In [6]:
pdfs = bsObje.findAll("a", {"class":"rightIconified iconDetalhe linkDownloadTeor"})

In [7]:
lista_arquivos = []

In [8]:
for name in pdfs:
    url = name.get('href')
    url = "http://www.camara.gov.br/proposicoesWeb/" + url
    posicao = url.find('filename=') + 9
    arquivo = (url[posicao:])
    arquivo = arquivo.replace('+', ' ')
    arquivo = arquivo.replace('/', '-')
    arquivo = arquivo.replace('%3D%3E', '=-')
    arquivo = arquivo + ".pdf"
    lista_arquivos.append([arquivo])
    urllib.request.urlretrieve(url, arquivo)

In [9]:
len(lista_arquivos)

1340

### Ler metadados de emendas